In [ ]:
library(scHOT)
library(dplyr)
library(ggplot2)
library(stringr)

In [ ]:
# Define helper function for barplot
makebarplot <- function(data, title, numpoints){
    data[['log10_pvalue']] <- -log10(data$pvalue)
    data_plot_df <- dplyr::arrange(data, desc(log10_pvalue))[1:numpoints,]
    data_plot_df$Description <- apply(str_split_fixed(data_plot_df$Description, '_', Inf), 1, paste, collapse=' ') %>% tolower()
    data_plot_df$Description <- str_wrap(data_plot_df$Description, width = 30)
    data_plot_df$Description <- factor(data_plot_df$Description, levels = rev(data_plot_df$Description))

    plot <- ggplot(data_plot_df, aes(x=log10_pvalue, y=Description, fill=Count)) +
    geom_bar(stat='identity') +
    xlab(bquote(~-Log[10] ~ 'p value')) +
    ylab(NULL) +
    ggtitle(title) +
    theme_bw() +
    scale_fill_gradientn(colours = jdb_palette('solar_rojos')[4:length(jdb_palette('solar_rojos'))]) +
    theme(plot.title = element_text(hjust = 0.5, size=17, face="bold"), axis.text.y = element_text(size=14),
    panel.border = element_blank(), panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(), axis.line = element_line(colour = "black"))
    return(plot)
}

In [ ]:
dataset <- 'embryo2_brain'

In [ ]:
# Import data

gex_res <- read.table(paste('/Users/ruiyanghe/Desktop/Uni/PartIII/Project/Nat_Biotech_Analysis/',dataset,'/data/final_gex_res.csv', collapse=''), header=TRUE, sep=',', row.names = 1)
combined_data <- read.table(paste('/Users/ruiyanghe/Desktop/Uni/PartIII/Project/Nat_Biotech_Analysis/',dataset,'/data/final_combined_data.csv'), header=TRUE, sep=',', row.names = 1)

force_res <- data.frame(pressure=combined_data$pressure,pressure_rank=rank(combined_data$pressure),
sm=combined_data$stresstensor_eigval1,sm_rank=rank(combined_data$stresstensor_eigval1))

scHOT_p_obj <- scHOT_buildFromMatrix(
    mat = as.matrix(gex_res),
    cellData = list(pseudotime = force_res$pressure_rank),
positionType = "trajectory",
positionColData = "pseudotime")

Analysis of weighted median